## TF-Core - eLU - Spitzer Calibration Data

This script show a simple example of using [tf.contrib.learn][1] library to create our model.

The code is divided in following steps:

 - Load CSVs data
 - Continuous features
 - Converting Data into Tensors
 - Selecting and Engineering Features for the Model
 - Defining The Regression Model
 - Training and Evaluating Our Model
 - Predicting output for test data

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, minmax_scale

from sklearn.metrics import r2_score

from tqdm import tqdm_notebook

from time import time
start0 = time()
plt.rcParams['figure.dpi'] = 300

In [ ]:
# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
# %matplotlib inline
# import matplotlib
# import matplotlib.pyplot as plt
# plt.rcParams['axes.labelsize'] = 14
# plt.rcParams['xtick.labelsize'] = 12
# plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
# PROJECT_ROOT_DIR = "."
# CHAPTER_ID = "ann"

# def save_fig(fig_id, tight_layout=True):
#     path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
#     print("Saving figure", fig_id)
#     if tight_layout:
#         plt.tight_layout()
#     plt.savefig(path, format='png', dpi=300)

## Load CSVs data

In [ ]:
nSkip = 20
spitzerDataRaw  = pd.read_csv('pmap_ch2_0p1s_x4_rmulti_s3_7.csv')

In [ ]:
PLDpixels = pd.DataFrame({key:spitzerDataRaw[key] for key in spitzerDataRaw.columns.values if 'pix' in key})
PLDpixels

In [ ]:
PLDnorm = np.sum(np.array(PLDpixels),axis=1)

In [ ]:
PLDpixels = (PLDpixels.T / PLDnorm).T
PLDpixels

In [ ]:
spitzerData = spitzerDataRaw.copy()
for key in spitzerDataRaw.columns: 
    if key in PLDpixels.columns:
        spitzerData[key] = PLDpixels[key]

In [ ]:
testPLD = np.array(pd.DataFrame({key:spitzerData[key] for key in spitzerData.columns.values if 'pix' in key}))
assert(not sum(abs(testPLD - np.array(PLDpixels))).all())
print('Confirmed that PLD Pixels have been Normalized to Spec')

In [ ]:
notFeatures     = ['flux', 'fluxerr', 'xerr', 'yerr', 'xycov', 't_cernox']

periodMax           = spitzerData['bmjd'].values.max() - spitzerData['bmjd'].values.min()
periodMin           = np.min(np.diff(spitzerData['bmjd'].values))
spitzerData['freq'] = np.linspace(np.pi/periodMax, 4*np.pi/periodMin, spitzerData['bmjd'].values.size)

feature_columns = spitzerData.drop(notFeatures,axis=1).columns.values
features        = spitzerData.drop(notFeatures,axis=1).values
labels          = spitzerData['flux'].values

In [ ]:
stdScaler = StandardScaler()

In [ ]:
features_scaled = stdScaler.fit_transform(features)
labels_scaled   = labels#stdScaler.fit_transform(labels[:,None]).ravel()

idx_valtest, idx_train = train_test_split(np.arange(labels_scaled.size), test_size=0.6, random_state=42)
idx_val, idx_test      = train_test_split(idx_valtest                  , test_size=0.5, random_state=42)

x_val   = features_scaled[idx_val]
x_test  = features_scaled[idx_test]
x_train = features_scaled[idx_train]

y_val   = labels_scaled[idx_val]
y_test  = labels_scaled[idx_test]
y_train = labels_scaled[idx_train]

y_val_err   = spitzerData['fluxerr'].values[idx_val]
y_test_err  = spitzerData['fluxerr'].values[idx_test]
y_train_err = spitzerData['fluxerr'].values[idx_train]

print(x_val.shape  , 'validation samples')
print(x_train.shape, 'train samples')
print(x_test.shape , 'test samples')

In [ ]:
train_df    = pd.DataFrame(np.c_[x_train, y_train], columns=list(feature_columns) + ['flux'])
test_df     = pd.DataFrame(np.c_[x_test , y_test ], columns=list(feature_columns) + ['flux'])
evaluate_df = pd.DataFrame(np.c_[x_val  , y_val  ], columns=list(feature_columns) + ['flux'])

We only take first 1000 rows for training/testing and last 500 row for evaluation.


This done so that this script does not consume a lot of kaggle system resources.

In [ ]:
# train_df = df_train_ori.head(1000)
# evaluate_df = df_train_ori.tail(500)

# test_df = df_test_ori.head(1000)

# MODEL_DIR = "tf_model_spitzer/withNormalization_drop50/relu"
# MODEL_DIR = "tf_model_spitzer/adamOptimizer_with_drop50/relu"
MODEL_DIR = "tf_model_spitzer/adamOptimizer/drop50/elu/"

print("train_df.shape = "   , train_df.shape)
print("test_df.shape = "    , test_df.shape)
print("evaluate_df.shape = ", evaluate_df.shape)

## Filtering Categorical and Continuous features

We store Categorical, Continuous and Target features names in different variables. This will be helpful in later steps.

In [ ]:
# categorical_features = [feature for feature in features if 'cat' in feature]
categorical_features  = []
continuous_features   = [feature for feature in train_df.columns]# if 'cat' in feature]
LABEL_COLUMN          = 'flux'

## Converting Data into Tensors

> When building a TF.Learn model, the input data is specified by means of an Input Builder function. This builder function will not be called until it is later passed to TF.Learn methods such as fit and evaluate. The purpose of this function is to construct the input data, which is represented in the form of Tensors or SparseTensors.

> Note that input_fn will be called while constructing the TensorFlow graph, not while running the graph. What it is returning is a representation of the input data as the fundamental unit of TensorFlow computations, a Tensor (or SparseTensor).

[More detail][2] on input_fn.

[2]: https://www.tensorflow.org/versions/r0.11/tutorials/input_fn/index.html#building-input-functions-with-tf-contrib-learn

In [ ]:
# Converting Data into Tensors
def input_fn(df, training = True):
    # Creates a dictionary mapping from each continuous feature column name (k) to
    # the values of that column stored in a constant Tensor.
    continuous_cols = {k: tf.constant(df[k].values)
                       for k in continuous_features}

    # Creates a dictionary mapping from each categorical feature column name (k)
    # to the values of that column stored in a tf.SparseTensor.
    # categorical_cols = {k: tf.SparseTensor(
    #     indices=[[i, 0] for i in range(df[k].size)],
    #     values=df[k].values,
    #     shape=[df[k].size, 1])
    #     for k in categorical_features}

    # Merges the two dictionaries into one.
    feature_cols = continuous_cols
    # feature_cols = dict(list(continuous_cols.items()) + list(categorical_cols.items()))
    
    if training:
        # Converts the label column into a constant Tensor.
        label = tf.constant(df[LABEL_COLUMN].values)

        # Returns the feature columns and the label.
        return feature_cols, label
    
    # Returns the feature columns    
    return feature_cols

def train_input_fn():
    return input_fn(train_df, training=True)

def eval_input_fn():
    return input_fn(evaluate_df, training=False)

def test_input_fn():
    return input_fn(test_df, training=False)

## Selecting and Engineering Features for the Model

We use tf.learn's concept of [FeatureColumn][FeatureColumn] which help in transforming raw data into suitable input features. 

These engineered features will be used when we construct our model.

[FeatureColumn]: https://www.tensorflow.org/versions/r0.11/tutorials/linear/overview.html#feature-columns-and-transformations

In [ ]:
engineered_features = []

for continuous_feature in continuous_features:
    engineered_features.append(
        tf.contrib.layers.real_valued_column(continuous_feature))

## Defining The Regression Model

Following is the simple DNNRegressor model. More detail about hidden_units, etc can be found [here][123].

**model_dir** is used to save and restore our model. This is because once we have trained the model we don't want to train it again, if we only want to predict on new data-set.

[123]: https://www.tensorflow.org/versions/r0.9/api_docs/python/contrib.learn.html#DNNRegressor

In [ ]:
config = tf.contrib.learn.RunConfig(tf_random_seed=42) # not shown in the config

In [ ]:
n_inputs  = features_scaled.shape[0]
n_features= features_scaled.shape[1]

In [ ]:
n_inputs  = x_train.shape[0]
n_features= x_train.shape[1]
n_inputs, n_features

In [ ]:
n_hidden1 = n_features
n_hidden2 = n_features
n_hidden3 = n_features
n_outputs = n_inputs   # because: regression

In [ ]:
reset_graph()

In [ ]:
with tf.name_scope('data'):
    X   = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
    y   = tf.placeholder(tf.float32, shape=(None), name="y")
    unc = tf.placeholder(tf.float32, shape=(None), name="unc")

In [ ]:
training = tf.placeholder_with_default(False, shape=(), name='training')

l1_reg       = 0.001
l2_reg       = 0.0001
dropout_rate = 0.5  # == 1 - keep_prob

X_bnorm= tf.layers.batch_normalization(X)
X_drop = tf.layers.dropout(X_bnorm, dropout_rate, training=training)

with tf.name_scope("dnn"):
    he_init      = tf.contrib.layers.variance_scaling_initializer()
    
    hidden1      = tf.layers.dense(X_drop , n_hidden1, name="hidden1", 
                        activation=tf.nn.elu, kernel_initializer=he_init,
                        kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(scale_l1=l1_reg, scale_l2=l2_reg))
    
    hidden1_bnorm= tf.layers.batch_normalization(hidden1, training=training, momentum=0.9)
    hidden1_drop = tf.layers.dropout(hidden1_bnorm, dropout_rate, training=training)
    
    hidden2      = tf.layers.dense(hidden1_drop, n_hidden2, name="hidden2", 
                        activation=tf.nn.elu, kernel_initializer=he_init,
                        kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(scale_l1=l1_reg, scale_l2=l2_reg))
    
    hidden2_bnorm= tf.layers.batch_normalization(hidden2, training=training, momentum=0.9)
    hidden2_drop = tf.layers.dropout(hidden2_bnorm, dropout_rate, training=training)
    
    hidden3      = tf.layers.dense(hidden2_drop, n_hidden3, name="hidden3", 
                        activation=tf.nn.elu, kernel_initializer=he_init,
                        kernel_regularizer=tf.contrib.layers.l1_l2_regularizer(scale_l1=l1_reg, scale_l2=l2_reg))
    
    hidden3_bnorm= tf.layers.batch_normalization(hidden3, training=training, momentum=0.9)
    hidden3_drop = tf.layers.dropout(hidden3_bnorm, dropout_rate, training=training)
    
    output  = tf.layers.dense(hidden3_drop, n_outputs, name="outputs")

In [ ]:
with tf.name_scope("loss"):
    def tf_nll(labels, output, uncs, coeff=1):
        error = output - labels
        return tf.reduce_sum(1 * (coeff * np.log(2*np.pi) + coeff * tf.log(uncs) + (0.5/uncs) * tf.pow(error, 2)))
    
    likelihood  = tf.reduce_mean(tf_nll(labels=y, output=output, uncs=unc))
    
    reg_losses  = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    loss        = tf.add_n([likelihood] + reg_losses, name="loss")
    
    mse         = tf.reduce_mean(tf.squared_difference(output, y, name="mse"))

In [ ]:
with tf.name_scope("eval"):
    r2_acc      = 1 - tf.reduce_mean(y-output) / tf.reduce_mean(y - tf.reduce_mean(y))
    rho2_acc    = 1 - tf.reduce_mean((y-output) / unc) / tf.reduce_mean((y-tf.reduce_mean(y)) / unc)
    accuracy    = tf.reduce_mean(tf.squared_difference(output, y, name="accuracy"))

In [ ]:
learning_rate = 0.001

# with tf.name_scope("train"):
optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss)

init = tf.global_variables_initializer()

In [ ]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

print(logdir)

In [ ]:
mse_summary  = tf.summary.scalar('MSE'       , mse )
loss_summary = tf.summary.scalar('loss'      , loss)
nll_summary  = tf.summary.scalar('likelihood', likelihood)
r2s_summary  = tf.summary.scalar('r2_acc'    , r2_acc)
p2s_summary  = tf.summary.scalar('rho2_acc'  , rho2_acc)
# hid1_hist    = tf.summary.histogram('hidden1', hidden1)
# hid2_hist    = tf.summary.histogram('hidden1', hidden1)
# hid3_hist    = tf.summary.histogram('hidden1', hidden1)

file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

In [ ]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
batch_size = 50
n_epochs   = 10
n_batches  = n_outputs // batch_size

In [ ]:
def fetch_batch(epoch, batch_index, batch_size, trainingNow=True):
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(y_train.size, size=batch_size)
    X_batch = x_train[indices]
    y_batch = y_train.reshape(-1, 1)[indices]
    u_batch = y_train_err.reshape(-1, 1)[indices]
    
    return {training: trainingNow, X: X_batch, y: y_batch, unc:unc_batch}

In [ ]:
def fetch_acc_batch(epoch, batch_index, batch_size, batch_size_mod=10, trainingNow=False):
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(y_val.size, size=batch_size_mod*batch_size)
    X_batch = x_val[indices]
    y_batch = y_val.reshape(-1, 1)[indices]
    u_batch = y_val_err.reshape(-1, 1)[indices]
    return {training: trainingNow, X: X_batch, y: y_batch, unc:unc_batch}

In [ ]:
def write_to_tensorboard(epoch, batch_index, feed_dict_now):
    step = epoch * n_batches + batch_index
    
    list_of_summaries = [nll_summary, mse_summary, loss_summary, r2s_summary, p2s_summary]
    
    for summary in list_of_summaries:
        file_writer.add_summary(summary=summary.eval(feed_dict=feed_dict_now), global_step=step)
    
    # summary_str_nll = nll_summary.eval(feed_dict=feed_dict_now)
    # summary_str_mse = mse_summary.eval(feed_dict=feed_dict_now)
    # summary_str_los = loss_summary.eval(feed_dict=feed_dict_now)
    # summary_str_r2s = r2s_summary.eval(feed_dict=feed_dict_now)
    # summary_str_p2s = p2s_summary.eval(feed_dict=feed_dict_now)
    # 
    # file_writer.add_summary(summary=summary_str_mse, global_step=step)
    # file_writer.add_summary(summary=summary_str_los, global_step=step)
    # file_writer.add_summary(summary=summary_str_r2s, global_step=step)
    # file_writer.add_summary(summary=summary_str_p2s, global_step=step)

In [ ]:
def print_update(feed_dict_now, epoch, batch_index, batch_size, n_batches, batch_size_mod=10):
    
    acc_test  = accuracy.eval(feed_dict=fetch_acc_batch(epoch, batch_index, batch_size))
    acc_train = accuracy.eval(feed_dict=feed_dict_now)
    
    step = epoch * n_batches + batch_index
    print(epoch, step, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in tqdm_notebook(range(n_epochs), total=n_epochs, desc='epoch'):
        for batch_index in tqdm_notebook(range(n_batches), total=n_batches, desc='batch'):
            feed_dict_now = fetch_batch(epoch, batch_index, batch_size)
            
            if batch_index % 10 == 0 and batch_index > 0: 
                write_to_tensorboard(epoch, batch_index, feed_dict_now)
                print_update(feed_dict_now, epoch, batch_index, batch_size, n_batches)
            
            sess.run(training_op, feed_dict=feed_dict_now)
        
        print_update(feed_dict_now, epoch, batch_index, batch_size, n_batches, batch_size_mod=100)
        # acc_test  = accuracy.eval(feed_dict=fetch_acc_batch(batch_index, batch_size_mod=100))
        # acc_train = accuracy.eval(feed_dict=feed_dict_now)
        # print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
    
    save_path = saver.save(sess, MODEL_DIR + "/spitzer_calibration_20_20_20_final.ckpt")

## Training and Evaluating Our Model

add progress bar through python `logging`

In [ ]:
import logging
logging.getLogger().setLevel(logging.INFO)

In [ ]:
# Training Our Model
nFitSteps = 100000
start = time()
wrap  = regressor.fit(input_fn=train_input_fn, steps=nFitSteps)
print('TF Regressor took {} seconds'.format(time()-start))

In [ ]:
# Evaluating Our Model
print('Evaluating ...')
results = regressor.evaluate(input_fn=test_input_fn, steps=1)

for key in sorted(results):
    print("{}: {}".format(key, results[key]))

print("Val Acc: {:.3f}".format((1-results['loss'])*100))

**Track Scalable Growth**

Shrunk data set to 23559 Training samples and 7853 Val/Test samples

| n_iters | time (s) | val acc | multicore | gpu |
|------------------------------------------------|
|  100    |   5.869  |  6.332 | yes | no |
|  200    |   6.380  | 13.178 | yes | no |
|  500    |   8.656  | 54.220 | yes | no |
|  1000   |  12.170  | 66.596 | yes | no |
|  2000   |  19.891  | 62.996 | yes | no |
|  5000   |  43.589  | 76.586 | yes | no |
|  10000  |  80.581  | 66.872 | yes | no |
|  20000  | 162.435  | 78.927 | yes | no |
|  50000  | 535.584  | 75.493 | yes | no |
|  100000 | 1062.656 | 73.162 | yes | no |

In [ ]:
nItersList = [100,200,500,1000,2000,5000,10000,20000,50000,100000]
rtimesList = [5.869, 6.380, 8.656, 12.170, 19.891, 43.589, 80.581, 162.435, 535.584, 1062.656]
valAccList = [6.332, 13.178, 54.220, 66.596, 62.996, 76.586, 66.872, 78.927, 75.493, 73.162]

In [ ]:
plt.loglog(nItersList, rtimesList,'o-');
plt.twinx()
plt.semilogx(nItersList, valAccList,'o-', color='orange');

## Predicting output for test data

Most of the time prediction script would be separate from training script (we need not to train on same data again) but I am providing both in same script here; as I am not sure if we can create multiple notebook and somehow share data between them in Kaggle.

In [ ]:
def de_median(x):
    return x - np.median(x)

In [ ]:
predicted_output = list(regressor.predict(input_fn=test_input_fn))
# x = list(predicted_output)

In [ ]:
r2_score(test_df['flux'].values,predicted_output)*100

In [ ]:
print('Full notebook took {} seconds'.format(time()-start0))